# Step 3: Testing

In [1]:
print('Importing libraries is in progress...')

# import the libraries
import tensorflow as tf
import keras
from keras.models import model_from_json
import os
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
import numpy as np
from numpy import concatenate
import json
import shutil
import h5py
import urllib
import requests
from sklearn.preprocessing import MinMaxScaler

print('Importing libraries completed successfully.')

Importing libraries is in progress...
Importing libraries completed successfully.


## Load the test data frame

In [2]:
#test_df = pd.read_csv('./data/preparedData/PM_test_prepared.csv', sep=",", header=0)
#test_df = pd.read_csv('./data/preparedData/PM_test_prepared_with_more_cycles_per_MID_4_1_4_0.csv', sep=",", header=0)
#test_df = pd.read_csv('./data/preparedData/PM_test_prepared_with_more_cycles_per_MID_full_0.csv', sep=",", header=0)
#test_df = pd.read_csv('./data/preparedData/PM_test_prepared_with_more_cycles_per_MID_full_1.csv', sep=",", header=0)
#test_df = pd.read_csv('./data/preparedData/PM_test_prepared_with_one_cycle_per_MID_2_1_2_0.csv', sep=",", header=0)
# A 92-es és a 93-mas gépnél mind a 30 label1=1 sor törölve lett, de már 1-et prediktált ezekre.
#test_df = pd.read_csv('./data/02_preparedData/PM_test_prepared_with_one_cycle_per_MID_2_1_2_0_messzebb_a_hibatol.csv', sep=",", header=0)
#   A 91-es gépnél 1 olyan sor van, ahol már a label1=1.
#   A 92-es gépnél mind a 30 label1=1 sor törölve lett, és előtte még 100 label1=0-ás sor.
#   A 93-mas gépnél mind a 30 label1=1 sor törölve lett, és előtte még 50 label1=0-ás sor.
#   A 100-as gépnél 1 olyan sor van, ahol már a label1=1.
#test_df = pd.read_csv('./data/preparedData/PM_test_prepared_with_one_cycle_per_MID_full_1.csv', sep=",", header=0)
#test_df.head(10)
test_df10 = pd.read_csv('./data/02_preparedData/PdM_test_prepared_failure1.csv', sep=",", header=0)
test_df20 = test_df10
test_df30 = test_df10
test_df40 = test_df10

In [3]:
#print('Column operations are in progress...')

# pick the feature columns 
# Sequence help order the observations in "time"
sequence_cols = ['cycle_norm']

# key columns group the machines
key_cols = ['machineID', 'cycle']

# Labels are what we're predicting.
label_cols = ['label1', 'label2', 'RUL']

# The scoreing data should not have labels... if we knew the label, 
# we wouldn'y need to predict.
score_df10 = test_df10.drop(label_cols, axis = 1)
score_df20 = test_df20.drop(label_cols, axis = 1)
score_df30 = test_df30.drop(label_cols, axis = 1)
score_df40 = test_df40.drop(label_cols, axis = 1)
#score_df.head(10)

#print('Column operations completed successfully.')

### Test init() and run() functions

In [4]:
print('Function init() defining is in progress...')

def init():
    # read in the model file
    #from keras.models import model_from_json
    global model10
    global model20
    global model30
    global model40

    # load json and create model
    #with open('./models/machineRULPredictor.json', 'r') as json_file:
    #    loaded_model_json = json_file.read()
    #    json_file.close()
    #    model = model_from_json(loaded_model_json)
    
    # load weights into new model
    #model.load_weights('./models/machineRULPredictor_weights.h5')
    #model.compile('sgd','mse',run_eagerly=True)
    
    model10 = keras.models.load_model('./models/machineRULfailure1Predictor.h5')
    model20 = keras.models.load_model('./models/machineRULfailure2Predictor.h5')
    model30 = keras.models.load_model('./models/machineRULfailure3Predictor.h5')
    model40 = keras.models.load_model('./models/machineRULfailure4Predictor.h5')

print('Function init() defining completed successfully.')

Function init() defining is in progress...
Function init() defining completed successfully.


In [5]:
print('Function run() defining is in progress...')

def run(score_input,model): 
    # Create the sequences
    sequence_length = 21
    sequence_cols = ['cycle_norm']
    key_cols = ['machineID', 'cycle']

    # Feature engineering
    #score_input = pd.read_csv('./data/preparedData/PM_test_prepared.csv', sep=",", header=0)
    #score_input=score_input.drop(label_cols, axis = 1)
    input_features = score_input.columns.values.tolist()
    #input_features = ['machineID','cycle','model','age','volt','rotate','pressure','vibration','cycle_norm']
    sensor_cols = [x for x in input_features if x not in set(key_cols)]
    sensor_cols = [x for x in sensor_cols if x not in set(sequence_cols)]

    # The time is sequenced along
    # This may be a silly way to get these column names, but it's relatively clear
    sequence_cols.extend(sensor_cols)
    
    seq_array = [score_input[score_input['machineID']==id][sequence_cols].values[-sequence_length:] 
                 for id in score_input['machineID'].unique() if len(score_input[score_input['machineID']==id]) >= sequence_length]

    seq_array = np.asarray(seq_array).astype(np.float32)
    try:
        prediction = model.predict(seq_array)
        pred = prediction.tolist()
        return(pred)
    except Exception as e:
        return(str(e))

print('Function run() defining completed successfully.')

Function run() defining is in progress...
Function run() defining completed successfully.


In [6]:
print(score_df10.machineID.unique())
print(score_df20.machineID.unique())
print(score_df30.machineID.unique())
print(score_df40.machineID.unique())

[ 89  91  92  93  98  99 100]
[ 89  91  92  93  98  99 100]
[ 89  91  92  93  98  99 100]
[ 89  91  92  93  98  99 100]


In [7]:
init()
prb10=run(score_df10,model10)
print(prb10)
prb20=run(score_df20,model20)
print(prb20)
prb30=run(score_df30,model30)
print(prb30)
prb40=run(score_df40,model40)
print(prb40)

1/1 [==============================] - 1s 1s/step
[[0.08544207364320755], [0.9706523418426514], [0.9860726594924927], [0.8993906378746033], [0.8351748585700989], [0.9823029637336731], [0.9967197775840759]]
1/1 [==============================] - 1s 1s/step
[[0.5472545623779297], [0.05812470242381096], [0.9383389949798584], [0.9306604862213135], [0.03157668188214302], [0.07582508772611618], [0.17734523117542267]]
1/1 [==============================] - 1s 1s/step
[[0.7291184663772583], [0.04084544628858566], [0.8668946027755737], [0.864972710609436], [0.04350130259990692], [0.47893455624580383], [0.8139830827713013]]
1/1 [==============================] - 1s 1s/step
[[0.1175309345126152], [0.14747807383537292], [0.3057021498680115], [0.33867019414901733], [0.13924351334571838], [0.054096292704343796], [0.058585964143276215]]
